In [ ]:
#change address to latitude and longtitude
import requests
import json
import csv
import pandas as pd
from tqdm import tqdm
# where to find original data "https://www.pavoterservices.pa.gov/Pages/PurchasePAFULLVoterExport.aspx?ID=%20PA_EXPORT_8454847762230114235637&Langcode=%20en-US"
# input sample csv "https://raw.githubusercontent.com/LyanRy/map-PA-voter/main/ADAMS_party.csv"

data = pd.read_csv("input.csv")

data['Latitude'] = ''
data['Longitude'] = ''

for index, row in tqdm(data.iterrows(), total=len(data)):
    address = row['Address']
    url = f"https://nominatim.openstreetmap.org/search?q={address}&format=json&limit=1"
    response = requests.get(url)
    
    # Check if response is successful
    if response.status_code == 200:
        result = json.loads(response.text)
        
        try:
            lat = result[0]["lat"]
            lng = result[0]["lon"]
        except (IndexError, KeyError):
            lat = ''
            lng = ''
            print(f"Error: {address}")
    else:
        print(f"Error: Request for {address} failed with status code {response.status_code}")
        # you may want to add additional handling for failed requests here

    data.at[index, 'Latitude'] = lat
    data.at[index, 'Longitude'] = lng

data.to_csv('output.csv', index=False)


In [ ]:
#use google api to geocode 
import requests
import json
import csv
import pandas as pd
from tqdm import tqdm


data = pd.read_csv('input.csv')


data['Latitude'] = ''
data['Longitude'] = ''


for index, row in tqdm(data.iterrows(), total=len(data)):

    address = row['Address']
    url = f"https://maps.googleapis.com/maps/api/geocode/json?address={address}&key=YourAPIkey"
    response = requests.get(url)
    result = json.loads(response.text)

    try:
 
        lat = result["results"][0]["geometry"]["location"]["lat"]
        lng = result["results"][0]["geometry"]["location"]["lng"]

    except (IndexError, KeyError):

        lat = ''
        lng = ''
        print(f"Error: {address}")


    data.at[index, 'Latitude'] = lat
    data.at[index, 'Longitude'] = lng



data.to_csv('add.csv', index=False)


In [ ]:
#Replace any party code other than 'R' and 'D' with 'Other'
import pandas as pd
# input sample csv "https://raw.githubusercontent.com/LyanRy/map-PA-voter/main/add.csv"
voter_data = pd.read_csv("input2.csv")
voter_data['Party Code'] = voter_data['Party Code'].apply(lambda x: 'Other' if x not in ['R', 'D'] else x)
voter_data.to_csv('output1.csv', index=False)


In [ ]:
#draw the map
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import contextily
# Load the voter data
#input sample csv "https://raw.githubusercontent.com/LyanRy/map-PA-voter/main/new.csv"

voter_data = pd.read_csv("input3.csv")
#set seaborn style and figure
sns.set(style="ticks", color_codes=True)
plt.figure(figsize=(100,80))
#create a jointplot using latitude and longtitude as axes and Party Code as hue
ax = sns.jointplot(x="Longitude", y="Latitude", data=voter_data, hue="Party Code", palette={"D": "blue", "R": "red", "Other": "black"}, kind="scatter", s=0.05)
#set the limits of the axes
plt.xlim([-77.6, -76.95])
plt.ylim([39.7, 40.1])
#add a basemap to the plot using contextily
contextily.add_basemap(ax.ax_joint, crs="EPSG:4326", source=contextily.providers.CartoDB.PositronNoLabels)
#save the plot as a PNG image
#output sample image "https://github.com/LyanRy/map-PA-voter/raw/main/map1.png"
plt.savefig("output.png", dpi=1600)

